## 0. Import Libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import math

from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import r2_score

from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf

from tqdm.notebook import tqdm

## 1. Set Up Constants

In [ ]:
TEST_SIZE = 0.2
SEQUENCE_LENGTH = 14
RANDOM_STATE_SEED = 42

RSI_ROLLING_WINDOW_SIZE = 14

## 2. Original Dataset

#### Read Dataset

In [ ]:
file_name = "SP500_forMacro.csv"

folder_name = "data"

folder_path = os.path.join(os.getcwd(), folder_name)

if os.path.isdir(folder_path):
    
    dataset_file_path = os.path.join(folder_path, file_name)
    
    df = pd.read_csv(dataset_file_path, index_col=False)
    
    print(f"Dataset has been read from {dataset_file_path}.")

else:
    print(f"Folder {folder_name} is not found at {folder_path}.")

#### Dataset Information and Summary

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.info()

In [ ]:
df.describe()

## 3. Dataset Pre-Processing

#### Missing Value Check

In [ ]:
df.isnull().sum()

In [ ]:
df['SP500'] = df['SP500'].ffill()

In [ ]:
df.isnull().sum()

In [ ]:
df.describe()

#### Set Index for Dataset

In [ ]:
df['observation_date'] = pd.to_datetime(df['observation_date'])

df.set_index('observation_date', inplace=True)

In [ ]:
df.head()

#### Removing First 'RSI Rolling Window Size' Days of Data

In [ ]:
df.head()

In [ ]:
# Dropout first n days of data due to missing data from GRU features (SMI)

df = df.iloc[(RSI_ROLLING_WINDOW_SIZE-1):]

#### Dataset Split

In [ ]:
# Columns = 'SP500'
y = df['SP500']

# Columns = ['SP500', 'SMA_10', 'RSI', 'MACD']
X = df.copy(deep=True)

In [ ]:
# Calculate starting index in the test dataset
start_idx = math.floor((1 - TEST_SIZE) * len(y))

# Obtain training and testing dataset for ARIMA
train_ARIMA = y[ :start_idx + SEQUENCE_LENGTH]
test_ARIMA = y[start_idx + SEQUENCE_LENGTH: ]

print(len(test_ARIMA))

## 4. Stationary Test

#### Plot SP500 Graph

In [ ]:
fig = plt.figure(figsize=(12, 8))

plt.plot(df['SP500'], color='blue', label='Price')

plt.title("SP500 Daily Price")
plt.legend()
plt.show()

#### Augmented Dickey-Fuller Test (ADF Test)

In [ ]:
## For p-value

"""
Confidence level = 0.95

Alpha value --> 0.05

Null hypothesis     : Series is Non-stationary 
Alternate hypothesis: Series is Stationary 

Decision logic: Reject null hypothesis if p-value < alpha
"""

In [ ]:
## For ADF statistics

"""
Alpha = 0.05

Decision Logic: If ADF statistics is lower than crtical value corresponding to alpha, reject null hypothesis
"""

In [ ]:
result = adfuller(df['SP500'])

#### Results Interpretation

In [ ]:
# result[0] --> ADF statistics
print(f"ADF Statistics: {result[0]}")   

# result[1] --> p-value 
print(f"P-value: {result[1]}")

print("\nCritical Values:")
for key, value in result[4].items():
    print(f"   {key}: {value}")

In [ ]:
## For p-value

"""
p-value = 0.9396044934918389
alpha   = 0.05

Comparison: p-value > alpha

Decision logic: There is insufficient evidence to reject null hypothesis

Conclusion: This series data is non-stationary
"""

In [ ]:
## For ADF statistics

"""
ADF Statistics      = -0.19084170734618847
Critical Value (5%) = -2.8630926485318433

Comparison: ADF statistics > Critical value

Decision logic: There is insufficient evidence to reject null hypothesis

Conclusion: This series data is non-stationary
"""

## 5. Hyperparameters of ARIMA (p, d, q)

#### Find Non-Seasonal Differencing Order Parameter 'd'

In [ ]:
first_diff_df = df['SP500'].diff().dropna()

second_diff_df = df['SP500'].diff().diff().dropna()

In [ ]:
# Original Dataset

fig = plt.figure(figsize=(12, 8))

plt.subplot(1, 2, 1)

plt.plot(df['SP500'])

plot_acf(df['SP500'], ax=plt.subplot(1, 2, 2), lags = len(df['SP500'])/2)

plt.show()

In [ ]:
# First-Order Differencing Dataset

fig = plt.figure(figsize=(12, 8))

plt.subplot(1, 2, 1)

plt.plot(first_diff_df)

plot_acf(first_diff_df, ax=plt.subplot(1, 2, 2))

plt.show()

In [ ]:
# Second-Order Differencing Dataset

fig = plt.figure(figsize=(12, 8))

plt.subplot(1, 2, 1)

plt.plot(second_diff_df)

plot_acf(second_diff_df, ax=plt.subplot(1, 2, 2))

plt.show()

In [ ]:
"""
Pick d = 1 as in the second-order differencing dataset: 

- Lag 1 clearly went from not negatively-significant to very signficant lag
- This indicates an overdifferencing has occured when d = 2
- Hence, we will pick the previous value of d where d = 1
"""

In [ ]:
# Check to see if first-order differencing makes the series stationary

result = adfuller(first_diff_df)

# result[0] --> ADF statistics
print(f"ADF Statistics: {result[0]}")   

# result[1] --> p-value 
print(f"P-value: {result[1]}")

print("\nCritical Values:")
for key, value in result[4].items():
    print(f"   {key}: {value}")

In [ ]:
## For p-value

"""
p-value = 2.0898364939452233e-22
alpha = 0.05

Comparison: p-value < alpha

Decision logic: There is sufficient evidence to reject null hypothesis

Conclusion: This series data is stationary
"""

In [ ]:
## For ADF statistics

"""
ADF Statistics      = -12.094102764805994
Critical Value (5%) = -2.8630926485318433

Comparison: ADF statistics < Critical value

Decision logic: There is sufficient evidence to reject null hypothesis

Conclusion: This series data is stationary
"""

In [ ]:
d = 1

#### Find Non-Seasonal AR Parameter 'p'

In [ ]:
plot_pacf(first_diff_df, alpha=0.05, title="PACF Plot", lags=52)

plt.show()

In [ ]:
plot_acf(first_diff_df, alpha=0.05, title="ACF Plot", lags=52)

plt.show()

In [ ]:
"""
From the PACF plot, we can see that:

Lag 9: First lag that is out of the significance limit boundary.

Lag 10: Not out of the signficance limit boundary by a lot.

This indicates after the significant lag 9, there is no more autoregressive (AR) structure.

This conclusion is also supported that PACF plot cuts off sharply after lag 9 where ACF tails off

AR term (p) = 9
"""

In [ ]:
p = 9

#### Identify Non-Seasonal Parameter 'q'

In [ ]:
plot_acf(first_diff_df, alpha=0.05, title="ACF plot", lags=52)

plt.show()

In [ ]:
"""
From the ACF plot, we can observe that --> 

Lag 0 is over the significance boundary.

After that, there are no more lags that cross the significance boundary.

This indcates that there are no dependence on current observation on any previous forecast errors.

As such, we shall pick q = 0.

MA term (q) = 0
"""

In [ ]:
q = 9

## 6. ARIMA Model

#### Model Setup

In [ ]:
model = ARIMA(train_ARIMA, order=(p, d, q))

#### Model Training

In [ ]:
result = model.fit(method_kwargs={'maxiter': 1000})

#### Training Residuals

In [ ]:
residuals = pd.DataFrame(result.resid)

In [ ]:
residuals.describe()

In [ ]:
residuals.plot(title="Residual Graph of ARIMA Dataset When Feed Full Dataset")

In [ ]:
residuals.plot(kind='kde', title="Residual Density Graph")

#### Testing ARIMA Model

In [ ]:
## Obtaining Error Value for Testing Dataset

arima_error_lst = []

test_pred_lst = []

history = [x for x in train_ARIMA]

for i in tqdm(range(0, len(test_ARIMA)), desc="Obtaining ARIMA Model error_lsts", leave=False):
    model = ARIMA(history,
                  order=(p, d, q))
    
    next_val = test_ARIMA[i]

    # Next training of ARIMA will include new value
    history.append(next_val)

    # Next training of ARIMA will discard earliest value to maintain window length
    history.pop(0)
    
    result = model.fit(method_kwargs={'maxiter': 1000})

    output = result.forecast(steps=1)

    # Output[0] --> Real value of prediction
    prediction = output[0]

    test_pred_lst.append(prediction)

    # Calculate error
    error = next_val - prediction

    arima_error_lst.append(error)

test_rmse = root_mean_squared_error(y_true=test_ARIMA,
                                    y_pred=test_pred_lst)

test_r2 = r2_score(y_true=test_ARIMA,
                   y_pred=test_pred_lst)

print(f'Test RMSE: {test_rmse:.3f}')

In [ ]:
print(len(arima_error_lst))

#### Model Evaluation Metrics

In [ ]:
test_rmse = root_mean_squared_error(y_true=test_ARIMA,
                                    y_pred=test_pred_lst)

print(f'Test RMSE: {test_rmse:.5f}')

#### Graph of Prediction vs Real Data Graph 

In [ ]:
## For Testing Predictions

fig = plt.figure(figsize=(12,8))

plt.plot(test_ARIMA.to_list(), color='green', label='Real Data')
plt.plot(test_pred_lst, color='red', label='Prediction Data')

graph_title = "Testing Dataset Prediction vs Real Data Graph"

plt.text(
    x=0.01, y=0.95,
    s=f"Test RMSE: {test_rmse}",
    transform=plt.gca().transAxes,
    fontsize=10,
)

plt.title(graph_title)
plt.legend()    
plt.show()

## 6. Save ARIMA Error Prediction

In [ ]:
arima_error_dic = {
    'Arima Error': arima_error_lst
}

arima_error_pd = pd.DataFrame(arima_error_dic)

folder_name = 'processed_data'

datafolder_filepath = os.path.join(os.getcwd(), folder_name)

if os.path.isdir(datafolder_filepath) == False:
    os.mkdir(datafolder_filepath)
    print(f"Folder {folder_name} created.")

arima_error_filename = "arima_error.csv"

arima_error_filepath = os.path.join(os.getcwd(), folder_name, arima_error_filename)

arima_error_pd.to_csv(arima_error_filepath, index=False)

print(f"Arima prediction error file successfully saved at:\n{arima_error_filepath}\n")